# Tayara Data Preprocessing: Offer Type by Text and Price

This notebook preprocesses the **full Tayara dataset** and robustly assigns `offer_type` (rent/sale) using both:
- **Text keyword extraction** (from the title)
- **Price-based fallback inference** (when text is ambiguous)

It produces a clean ML/EDA-ready CSV with all features and the best possible split of 'rent' vs. 'sale'.

## 1. Setup & Load Data

In [1]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('tayara_real_estate_p1_p317.csv')
print(f"Loaded {len(df)} rows.")
df.head()

Loaded 9510 rows.


,page,title,price,location,listing_date,url
0,1,عقاري تجاري,NaN,Gafsa,a minute ago,https://www.tayara.tn/item/magasins%2c-commerc...
1,1,Studio Luxe à louer,270 DT,Ariana,2 minutes ago,https://www.tayara.tn/item/appartements/ariana...
2,1,📍ENNASR 2 : APPARTEMENT S+3 A LOUER,1450 DT,Ariana,2 minutes ago,https://www.tayara.tn/item/appartements/ariana...
3,1,A louer s1 meublé manar1,800 DT,Tunis,3 minutes ago,https://www.tayara.tn/item/appartements/tunis/...
4,1,📍 NOUVELLE ARIANA : BUREAU H+3 SPACIEUX À LOUER,1400 DT,Ariana,3 minutes ago,https://www.tayara.tn/item/bureaux-et-plateaux...


## 2. Remove Duplicates, Standardize Columns

In [2]:
df = df.rename(columns=lambda x: x.strip().lower())
df = df.drop_duplicates('url').reset_index(drop=True)
df.head()

,page,title,price,location,listing_date,url
0,1,عقاري تجاري,NaN,Gafsa,a minute ago,https://www.tayara.tn/item/magasins%2c-commerc...
1,1,Studio Luxe à louer,270 DT,Ariana,2 minutes ago,https://www.tayara.tn/item/appartements/ariana...
2,1,📍ENNASR 2 : APPARTEMENT S+3 A LOUER,1450 DT,Ariana,2 minutes ago,https://www.tayara.tn/item/appartements/ariana...
3,1,A louer s1 meublé manar1,800 DT,Tunis,3 minutes ago,https://www.tayara.tn/item/appartements/tunis/...
4,1,📍 NOUVELLE ARIANA : BUREAU H+3 SPACIEUX À LOUER,1400 DT,Ariana,3 minutes ago,https://www.tayara.tn/item/bureaux-et-plateaux...


## 3. Price Extraction

In [3]:
def extract_price_amount(text):
    if pd.isna(text): return np.nan
    m = re.search(r'([\d\s]+)', str(text))
    if m: return float(m.group(1).replace(' ', ''))
    return np.nan

def extract_price_currency(text):
    if pd.isna(text): return None
    m = re.search(r'(DT|TND|EUR|USD)', str(text).upper())
    return m.group(1) if m else 'DT'

df['price_amount'] = df['price'].apply(extract_price_amount)
df['price_currency'] = df['price'].apply(extract_price_currency)
df[['price','price_amount','price_currency']].head(5)

,price,price_amount,price_currency
0,NaN,NaN,None
1,270 DT,270.0,DT
2,1450 DT,1450.0,DT
3,800 DT,800.0,DT
4,1400 DT,1400.0,DT


## 4. Surface Area and Rooms Extraction

In [4]:
def extract_surface(text):
    if pd.isna(text): return np.nan
    m = re.search(r'(\d{2,4})\s*(m²|m2|m\b)', str(text).replace('.', ''))
    if m:
        return float(m.group(1))
    return np.nan
df['surface_m2'] = df['title'].apply(extract_surface)

def extract_rooms(text):
    if pd.isna(text): return np.nan
    m = re.search(r'S\s*\+\s*(\d+)', str(text), re.IGNORECASE)
    if m:
        return int(m.group(1))
    return np.nan
df['rooms'] = df['title'].apply(extract_rooms)
df.head()

,page,title,price,location,listing_date,url,price_amount,price_currency,surface_m2,rooms
0,1,عقاري تجاري,NaN,Gafsa,a minute ago,https://www.tayara.tn/item/magasins%2c-commerc...,NaN,None,NaN,NaN
1,1,Studio Luxe à louer,270 DT,Ariana,2 minutes ago,https://www.tayara.tn/item/appartements/ariana...,270.0,DT,NaN,NaN
2,1,📍ENNASR 2 : APPARTEMENT S+3 A LOUER,1450 DT,Ariana,2 minutes ago,https://www.tayara.tn/item/appartements/ariana...,1450.0,DT,NaN,3.0
3,1,A louer s1 meublé manar1,800 DT,Tunis,3 minutes ago,https://www.tayara.tn/item/appartements/tunis/...,800.0,DT,NaN,NaN
4,1,📍 NOUVELLE ARIANA : BUREAU H+3 SPACIEUX À LOUER,1400 DT,Ariana,3 minutes ago,https://www.tayara.tn/item/bureaux-et-plateaux...,1400.0,DT,NaN,NaN


## 5. Property Type Extraction

In [5]:
property_types = ['villa','appartement','studio','terrain','bureau','duplex','local','maison']
def extract_type(text):
    if pd.isna(text): return np.nan
    for t in property_types:
        if re.search(t, str(text), re.IGNORECASE):
            return t.lower()
    return np.nan
df['property_type'] = df['title'].apply(extract_type)
df.head()

,page,title,price,location,listing_date,url,price_amount,price_currency,surface_m2,rooms,property_type
0,1,عقاري تجاري,NaN,Gafsa,a minute ago,https://www.tayara.tn/item/magasins%2c-commerc...,NaN,None,NaN,NaN,NaN
1,1,Studio Luxe à louer,270 DT,Ariana,2 minutes ago,https://www.tayara.tn/item/appartements/ariana...,270.0,DT,NaN,NaN,studio
2,1,📍ENNASR 2 : APPARTEMENT S+3 A LOUER,1450 DT,Ariana,2 minutes ago,https://www.tayara.tn/item/appartements/ariana...,1450.0,DT,NaN,3.0,appartement
3,1,A louer s1 meublé manar1,800 DT,Tunis,3 minutes ago,https://www.tayara.tn/item/appartements/tunis/...,800.0,DT,NaN,NaN,NaN
4,1,📍 NOUVELLE ARIANA : BUREAU H+3 SPACIEUX À LOUER,1400 DT,Ariana,3 minutes ago,https://www.tayara.tn/item/bureaux-et-plateaux...,1400.0,DT,NaN,NaN,bureau


## 6. Offer Type (by Keyword in Title, then Fallback by Price)

In [6]:
def extract_offer_type(text):
    if pd.isna(text): return 'unknown'
    t = str(text).lower()
    for_sale = ['à vendre', 'a vendre', 'vente', 'for sale', 'بيع', 'sell']
    for_rent = ['à louer', 'a louer', 'location', 'for rent', 'louer', 'للإيجار', 'كراء', 'lease', 'rent']
    if any(word in t for word in for_sale):
        return 'sale'
    if any(word in t for word in for_rent):
        return 'rent'
    return 'unknown'

df['offer_type'] = df['title'].apply(extract_offer_type)

# Secondary: try property type string if still unknown
mask = df['offer_type'] == 'unknown'
df.loc[mask, 'offer_type'] = df.loc[mask, 'property_type'].apply(
    lambda x: 'rent' if pd.notna(x) and 'location' in x else ('sale' if pd.notna(x) and ('vente' in x or 'vendre' in x) else 'unknown'))

# Final fallback: Use price
def infer_offer_type_by_price(row):
    if row['offer_type'] != 'unknown':
        return row['offer_type']
    price = row['price_amount']
    if pd.isna(price):
        return 'unknown'
    # Heuristic: <10,000 DT (and usually >50) is rent; >20,000 DT is sale
    if price < 10000:
        return 'rent'
    if price > 20000:
        return 'sale'
    return 'unknown'

df['offer_type'] = df.apply(infer_offer_type_by_price, axis=1)
print(df['offer_type'].value_counts())
df[['title','price','price_amount','offer_type']].sample(10)

offer_type
rent       4997
sale       4001
unknown     479
Name: count, dtype: int64


,title,price,price_amount,offer_type
1150,terrain pour immeuble riadh el andalos,650000 DT,650000.0,sale
5214,Maison R+2 inachevée à cité mirdès Kelibia,380000 DT,380000.0,sale
8317,A LOUER S+1 HAUT STANDING MEUBLÉE A HAY EL WAHAT,1300 DT,1300.0,rent
3137,Terrain d’habitation 264 m² à cité riadh 2 Kel...,170000 DT,170000.0,sale
5568,🏠 À VENDRE | S+2 & S+3 avec jardins – Haut Sta...,360000 DT,360000.0,sale
5850,loyer,160 DT,160.0,rent
6929,A louer un étage de villa au Bardo sur rue 20 ...,1000 DT,1000.0,rent
3912,terrain a vendre sousse bouhsina,730 DT,730.0,sale
4542,Bureau en 3 espaces-90m²-Montplaisir-IFCM223,1800 DT,1800.0,rent
2046,A louer un appartement S+3 à Boumhel Côté Ben ...,700 DT,700.0,rent


## 7. Location Standardization

In [7]:
df['location'] = df['location'].str.strip().str.capitalize()
df['location'] = df['location'].replace({'tunis ': 'Tunis', 'ariana ': 'Ariana'})
df['location'].value_counts().head(10)

location
Tunis         3594
Ariana        1578
Sousse        1060
Nabeul         937
Ben arous      819
Sfax           448
Bizerte        225
Monastir       190
La manouba     166
Mahdia         134
Name: count, dtype: int64

## 8. Save Cleaned Data (with offer_type)

In [ ]:
# Remove rows with NO key extracted data
before = len(df)
df = df[~(
    df['price_amount'].isna() & df['surface_m2'].isna() & 
    df['rooms'].isna() & df['property_type'].isna()
)]
print(f"Dropped {before-len(df)} mostly empty rows.")

out_cols = ['page','title','price','price_amount','price_currency','location',
            'surface_m2','rooms','property_type','offer_type','listing_date','url']
df[out_cols].to_csv('tayara_real_estate_cleaned_offer_by_text_and_price.csv', index=False)
print(f"Saved to tayara_real_estate_cleaned_offer_by_text_and_price.csv, rows: {len(df)}")

Dropped 423 mostly empty rows.
Saved to tayara_real_estate_cleaned_offer_by_text_and_price.csv, rows: 9054
